### Imports ###

In [230]:
import os,sys
import http.client
import json
from datetime import datetime
import pandas as pd

'/root/feetball'

### Statics ###

Folders

In [381]:
resources=os.path.join(os.path.abspath('..'),'resources','csv')

API

In [464]:
# Football API
FOOTBALL_API_KEY = 'cbc491cd36f8a38ff058fe85f3265b8b'
URL = 'https://v3.football.api-sports.io'
api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

# FUT API
FUT_API_KEY = 'f64149b1-ecbe-46c1-9b7f-60ae79dac51e'

Dates

In [220]:
today= datetime.strftime(datetime.now(),'%Y-%m-%d')

Football

In [265]:
top_leagues = [2,3,39,61,78,88,135,140]

#### Base data ####

Leagues

In [250]:
# Set target file for leagues
target_file = os.path.join(resources,'leagues.csv')

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    api_football = http.client.HTTPSConnection("v3.football.api-sports.io")
    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': FOOTBALL_API_KEY
        }
    params = f""

    api_football.request("GET", f"/leagues?{params}", headers=headers)

    res = api_football.getresponse()
    data = res.read()
    data_json = json.loads(data)
    
    all_rows=[]
    for i in j['response']:
        row={}
        row['id'] = i['league']['id']
        row['name'] = i['league']['name']
        row['country'] = i['country']['name']
        row['first_year'] = i['seasons'][0]['year']
        row['current']= ''
        for ii in i['seasons']:
            if ii['current']==True:
                row['current'] = ii['year']
        
        all_rows.append(row)

    leagues = pd.DataFrame(all_rows)
    leagues.to_csv(target_file,index=False)
else:
    # Otherwise read the file
    leagues=pd.read_csv(target_file)

Teams

In [384]:
# Set target file for leagues
target_file = os.path.join(resources,'teams.csv')

# Target connection
api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    all_rows = []
    for i,row in leagues.iterrows():
        league_id = row['id']
        league_season = row['current']
        league_name = row['name']
        if league_id in top_leagues:
            headers = {
                'x-rapidapi-host': "v3.football.api-sports.io",
                'x-rapidapi-key': FOOTBALL_API_KEY
                }
            params = f"league={league_id}&&season={league_season}"

            api_football.request("GET", f"/teams?{params}", headers=headers)

            res = api_football.getresponse()
            data = res.read()
            data_json = json.loads(data)
            for i in data_json['response']:
                row = {}
                row['id'] = i['team']['id']
                row['name'] = i['team']['name']
                row['code'] = i['team']['code']
                row['league_id'] = league_id
                row['league_name'] = league_name
                row['league_season'] = league_season
                row['venue_id'] = i['venue']['id']
                row['venue_name'] = i['venue']['name']
                row['venue_capacity'] = i['venue']['capacity']
                all_rows.append(row)

    teams = pd.DataFrame(all_rows)
    teams.to_csv(target_file,index=False)
else:
    # Otherwise read the file
    teams=pd.read_csv(target_file)

Fixtures

In [303]:
api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': FOOTBALL_API_KEY
    }
params = f"date={today}"

api_football.request("GET", f"/fixtures?{params}", headers=headers)

res = api_football.getresponse()
data = res.read()
data_json = json.loads(data)

all_rows = []
for i in data_json['response']:
    row = {}
    row['status'] = i['fixture']['status']['long']
    row['date'] = i['fixture']['date']
    row['timestamp'] = i['fixture']['timestamp']
    row['id'] = i['fixture']['id']
    row['league_id'] = i['league']['id']
    row['league_name'] = i['league']['name']
    row['home_id'] = i['teams']['home']['id']
    row['home_name'] = i['teams']['home']['name']
    row['away_id'] = i['teams']['away']['id']
    row['away_name'] = i['teams']['away']['name']
    all_rows.append(row)

fixtures = pd.DataFrame(all_rows)
fixtures['date'] =  pd.to_datetime(fixtures['date'])
fixtures = fixtures[fixtures['league_id'].isin(top_leagues)]

Target game

In [342]:
target_game = 871344

Getting selected fixture

In [399]:
home_team = fixtures[fixtures['id']==target_game]['home_id'].min()
away_team = fixtures[fixtures['id']==target_game]['away_id'].min()
league = fixtures[fixtures['id']==target_game]['league_id'].min()
season = teams[teams['league_id']==league]['league_season'].min()

this_match = [
    {'team':'home','id':home_team},
    {'team':'away','id':away_team},
                ]

In [401]:
for i in this_match:
    print(i['team'])

home
away


Get the two teams stats

In [431]:
for t in this_match:
    id = t['id']
    team = t['team']
    api_football = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': FOOTBALL_API_KEY
        }
    params = f"season={season}&team={id}&league={league}"

    api_football.request("GET", f"/teams/statistics?{params}", headers=headers)

    res = api_football.getresponse()
    data = res.read()
    data_json = json.loads(data)

    all_rows = []
    i= data_json['response']
    row ={}
    row['id']=i['team']['id']
    row['name']=i['team']['name']
    row['form']=i['form']
    row['played']=i['fixtures']['played']['total']
    row['played_home']=i['fixtures']['played']['home']
    row['played_away']=i['fixtures']['played']['away']
    row['wins']=i['fixtures']['wins']['total']
    row['wins_home']=i['fixtures']['wins']['home']
    row['wins_away']=i['fixtures']['wins']['away']
    row['draws']=i['fixtures']['draws']['total']
    row['draws_home']=i['fixtures']['draws']['home']
    row['draws_away']=i['fixtures']['draws']['away']
    row['loses']=i['fixtures']['loses']['total']
    row['loses_home']=i['fixtures']['loses']['home']
    row['loses_away']=i['fixtures']['loses']['away']
    row['goals_for']=i['goals']['for']['total']['total']
    row['goals_for_home']=i['goals']['for']['total']['home']
    row['goals_for_away']=i['goals']['for']['total']['away']
    row['goals_for_avg']=i['goals']['for']['average']['total']
    row['goals_for_avg_home']=i['goals']['for']['average']['home']
    row['goals_for_avg_away']=i['goals']['for']['average']['away']
    row['goals_for_min_15']=i['goals']['for']['minute']['0-15']['percentage']
    row['goals_for_min_30']=i['goals']['for']['minute']['16-30']['percentage']
    row['goals_for_min_45']=i['goals']['for']['minute']['31-45']['percentage']
    row['goals_for_min_60']=i['goals']['for']['minute']['46-60']['percentage']
    row['goals_for_min_75']=i['goals']['for']['minute']['61-75']['percentage']
    row['goals_for_min_90']=i['goals']['for']['minute']['76-90']['percentage']
    row['goals_against']=i['goals']['against']['total']['total']
    row['goals_against_home']=i['goals']['against']['total']['home']
    row['goals_against_away']=i['goals']['against']['total']['away']
    row['goals_against_avg']=i['goals']['against']['average']['total']
    row['goals_against_avg_home']=i['goals']['against']['average']['home']
    row['goals_against_avg_away']=i['goals']['against']['average']['away']
    row['goals_against_min_15']=i['goals']['against']['minute']['0-15']['percentage']
    row['goals_against_min_30']=i['goals']['against']['minute']['16-30']['percentage']
    row['goals_against_min_45']=i['goals']['against']['minute']['31-45']['percentage']
    row['goals_against_min_60']=i['goals']['against']['minute']['46-60']['percentage']
    row['goals_against_min_75']=i['goals']['against']['minute']['61-75']['percentage']
    row['goals_against_min_90']=i['goals']['against']['minute']['76-90']['percentage']
    row['clean_sheet']=i['clean_sheet']['total']
    row['clean_sheet_home']=i['clean_sheet']['home']
    row['clean_sheet_away']=i['clean_sheet']['away']
    row['failed_to_score']=i['failed_to_score']['total']
    row['failed_to_score_home']=i['failed_to_score']['home']
    row['failed_to_score_away']=i['failed_to_score']['away']

    all_rows.append(row)

    if team=='home':
        home = pd.DataFrame(all_rows).melt()
    else:
        away = pd.DataFrame(all_rows).melt()

home['team'] = 'home'
away['team'] = 'away'
match_results = pd.concat([home,away])
match_results = match_results.pivot_table(index='variable',columns='team',values='value',aggfunc='sum').reset_index()

Last few games

In [458]:
for i in [1,3,5,10,20]:
    row = pd.DataFrame({
        'variable':f'last {i} games - wins',
        'home': match_results.query('variable=="form"')['home'].min()[:i].count('W'),
        'away': match_results.query('variable=="form"')['away'].min()[:i].count('W')
    },index=[len(match_results)])
    match_results = pd.concat([match_results,row])

### FUT ###

In [521]:
# Set target file for leagues
target_file = os.path.join(resources,'players.csv')

# If the leagues file is missing, then get it again from the API
if os.path.isfile(target_file)==False:
    api_fut = http.client.HTTPSConnection("futdb.app")
    n=1
    total_pages = 2
    headers = {
        'accept': "application/json",
        'X-AUTH-TOKEN': FUT_API_KEY
        }
    all_rows = []
    while n<=total_pages:
        params = f"page={n}"

        api_fut.request("GET", f"/api/players?{params}", headers=headers)

        res = api_fut.getresponse()
        data = res.read()
        data_json = json.loads(data)
        if n==1:
            total_pages = data_json['pagination']['pageTotal']

        for i in data_json['items']:
            row = {}
            row['id'] = i['id']
            row['firstName'] = i['firstName']
            row['lastName'] = i['lastName']
            row['name'] = i['name']
            row['height'] = i['height']
            row['weight'] = i['weight']
            row['birthdate'] = i['birthDate']
            row['position'] = i['position']
            row['skillMoves'] = i['skillMoves']
            row['weakFoot'] = i['weakFoot']
            row['totalStats'] = i['totalStats']
            row['rating'] = i['rating']
            row['ratingAverage'] = i['ratingAverage']
            row['pace'] = i['pace']
            row['shooting'] = i['shooting']
            row['passing'] = i['passing']
            row['dribbling'] = i['dribbling']
            row['defending'] = i['defending']
            row['physicality'] = i['physicality']
            all_rows.append(row)

        n+=1
    players = pd.DataFrame(all_rows)
    players.to_csv(target_file,index=False)
else:
    players = pd.read_csv(target_file)

In [524]:
data_json['items']

[{'id': 3342,
  'resourceId': 261876,
  'resourceBaseId': 261876,
  'futBinId': 46512,
  'futWizId': 7609,
  'firstName': 'Conan',
  'lastName': 'Noonan',
  'name': 'Conan Noonan',
  'commonName': 'Noonan',
  'height': 178,
  'weight': 0,
  'birthDate': '2003-01-17',
  'age': 20,
  'league': 65,
  'nation': 25,
  'club': 306,
  'rarity': 0,
  'traits': [],
  'specialities': [],
  'position': 'ST',
  'skillMoves': 2,
  'weakFoot': 3,
  'foot': 'Right',
  'attackWorkRate': 'Med',
  'defenseWorkRate': 'Med',
  'totalStats': 239,
  'totalStatsInGame': 1177,
  'color': 'bronze',
  'rating': 47,
  'ratingAverage': 39,
  'pace': 53,
  'shooting': 47,
  'passing': 36,
  'dribbling': 47,
  'defending': 17,
  'physicality': 39,
  'paceAttributes': {'acceleration': 53, 'sprintSpeed': 53},
  'shootingAttributes': {'positioning': 48,
   'finishing': 50,
   'shotPower': 44,
   'longShots': 44,
   'volleys': 36,
   'penalties': 56},
  'passingAttributes': {'vision': 44,
   'crossing': 32,
   'freeKic

In [497]:
data_json['items'][10].keys()

dict_keys(['id', 'resourceId', 'resourceBaseId', 'futBinId', 'futWizId', 'firstName', 'lastName', 'name', 'commonName', 'height', 'weight', 'birthDate', 'age', 'league', 'nation', 'club', 'rarity', 'traits', 'specialities', 'position', 'skillMoves', 'weakFoot', 'foot', 'attackWorkRate', 'defenseWorkRate', 'totalStats', 'totalStatsInGame', 'color', 'rating', 'ratingAverage', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physicality', 'paceAttributes', 'shootingAttributes', 'passingAttributes', 'dribblingAttributes', 'defendingAttributes', 'physicalityAttributes', 'goalkeeperAttributes'])